### Importing packages and bibs

In [1]:
import pandas as pd

from mogestpy.quantity.Hydrological import SMAP
from spotpy.objectivefunctions import mae, rmse, nashsutcliffe, kge, pbias
from scipy.optimize import Bounds

#### Pandas Plotting Backend

Plotly creates html graphs that are useful for data visualization

In [2]:
pd.options.plotting.backend = "plotly"

Loading data into a Pandas DataFrame

In [3]:
df = pd.read_excel(r'D:\Dados\smap_jundiai.xlsx')
display(df)

,ETP,P,Qobs,Qplan
0,5.266667,0.000000,1.69,-0.323000
1,5.266667,0.000000,0.06,-0.319765
2,5.266667,1.100000,0.41,-0.316562
3,5.266667,0.000000,0.06,-0.313391
4,5.266667,0.000000,1.94,-0.310251
...,...,...,...,...
2118,3.233333,0.000000,0.11,0.028231
2119,3.233333,0.283333,0.09,0.025673
2120,3.233333,0.003333,0.02,0.024410
2121,3.233333,0.496667,0.01,0.023720


Leaving watershed input data as default except for drainage area because .Calibrate() will override these values.

In [4]:
bacia = SMAP.SMAP.Basin(AD=109.08)
ponto = SMAP.SMAP.Point(df['P'], df['ETP'])
modelo = SMAP.SMAP(ponto, bacia)

In [5]:
result = modelo.Calibrate(df.Qobs, optimization_engine='de')

In [6]:
print(bacia)

SMAP Basin Object:
  Str = 220.50030836954716,
  Crec = 3.9548929396826527,
  Capc = 39.227910162679066,
  kkt = 103.64064573633222,
  k2t = 0.9544683059791361,
  Ai = 2.00000001,
  TUin = 0.2794515361707833,
  EBin = 12.472074298397635,
  AD = 109.08


In [7]:
df['Qcodigo'] = modelo.Q

In [8]:
result2 = modelo.Calibrate(df['Qobs'], objective_function='kge')

In [9]:
result2

      fun: -0.6530595964585548
 hess_inv: <8x8 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 1.07707244e-03,  0.00000000e+00, -4.25109945e-03,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  2.55351294e-07,  1.58313807e-02])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 432
      nit: 37
     njev: 48
   status: 0
  success: True
        x: array([100.        ,  10.        ,   1.        ,   0.        ,
        40.        , 105.        ,   0.95254134,   2.        ])

In [10]:
df['Qfo-teste'] = modelo.Q

In [11]:
def funcao_objetivo(eval, sim):
    somatorio = 0
    for i, j in zip(eval, sim):
        somatorio += (i - j) ** 2
    return somatorio

In [12]:
result_test = modelo.Calibrate(df.Qobs, objective_function=funcao_objetivo, optimization_engine='differential_evolution')

KeyboardInterrupt: 

In [ ]:
result_test

     fun: 16185.100806196497
 message: 'Optimization terminated successfully.'
    nfev: 2352
     nit: 18
 success: True
       x: array([2.32758597e+02, 6.45738003e+00, 2.01220657e-01, 3.61557553e+00,
       3.95057995e+01, 1.04453889e+02, 9.34341505e-01, 2.00916733e+00])

In [ ]:
print(bacia)

SMAP Basin Object:
  Str = 232.75860315434687,
  Crec = 6.457380076609695,
  Capc = 39.50579958012301,
  kkt = 104.45388941250211,
  k2t = 0.9343415361995924,
  Ai = 2.0091673537087633,
  TUin = 0.2012206597726131,
  EBin = 3.6155755821769953,
  AD = 109.08


In [ ]:
df['Qcustom_fo'] = modelo.Q

In [ ]:
bacia2=SMAP.SMAP.Basin(
    Str = 235.00569097847588,
    Crec = 11.489898491508313,
    TUin = 0.8544213137461782,
    EBin = 10.965839303606261,
    Capc = 40.02608612021428,
    kkt = 79.03366335608037,
    k2t = 0.9532324848266956,
    Ai = 2.0253907074478206,
    AD=109.08
)

In [ ]:
modelo2 = SMAP.SMAP(ponto, bacia2)

In [ ]:
modelo2.Calibrate(df.Qobs, objective_function='kge')

      fun: -0.35239627328026346
 hess_inv: <8x8 LbfgsInvHessProduct with dtype=float64>
      jac: array([13620605.33642578, 13620477.5759475 , 13620478.63460565,
       13620478.70304528, 13620477.57608191, 13620487.25401939,
       13620478.53183559, 13620478.79531523])
  message: 'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 189
      nit: 0
     njev: 21
   status: 2
  success: False
        x: array([1.05e+03, 1.00e+01, 5.00e-01, 0.00e+00, 4.00e+01, 1.05e+02,
       2.00e-01, 3.50e+00])

In [ ]:
df['Calib_DE'] = modelo2.Q

In [ ]:
df[['Qobs', 'Qplan', 'Qcodigo', 'Qcustom_fo', 'Calib_DE']].plot()

NameError: name 'df' is not defined

In [ ]:
bacia = SMAP.SMAP.Basin(
    Str = 232.75860315434687,
    Crec = 6.457380076609695,
    Capc = 39.50579958012301,
    kkt = 104.45388941250211,
    k2t = 0.9343415361995924,
    Ai = 2.0091673537087633,
    TUin = 0.2012206597726131,
    EBin = 3.6155755821769953,
    AD = 109.08
)

In [ ]:
bacia = SMAP.SMAP.Basin(    
    Str = 232.75860315434687,
    Crec = 6.457380076609695,
    Capc = 39.50579958012301,
    kkt = 104.45388941250211,
    k2t = 0.9343415361995924,
    Ai = 2.0091673537087633,
    TUin = 0.2012206597726131,
    EBin = 3.6155755821769953,
    AD = 109.08
)